In [16]:
!pip install pyedflib

In [17]:
# %% [markdown]
"""
# ChronoNet-Inspired EEG Brain Map Classification (Merged .eea and .edf)
This script:
- Reads EEG data from two sources: .eea files (e.g., from "/kaggle/input/mhrc-dataset-data") and .edf files (e.g., from "/kaggle/input/schizophrenia").
- Computes PSD, Spectral Entropy (SE), and Differential Entropy (DE) features for four frequency bands (theta, alpha, beta, gamma).
- Creates 2D brain maps (8×9) for each band.
- Merges the two datasets according to your project logic.
- Augments the data:
    - For target class 1, each sample is augmented (n_augments=4 per sample).
    - Then, class 0 is augmented until the final number of class 0 samples is twice that of class 1.
- Stacks the three feature types for each band into ChronoNet‐compatible inputs (each sample becomes a sequence of 4 time steps, each with 216 features).
- Builds and trains a ChronoNet‐inspired model (using parallel 1D conv layers with varying kernel sizes and GRU layers).
- Plots training curves and evaluation metrics.
"""

# %% 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pyedflib
from scipy.signal import welch
from scipy.stats import entropy
from zipfile import ZipFile
from tensorflow.keras import layers, models, Model, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split

# %% [markdown]
"""
## 1. Functions for Reading Files and Computing Features
These functions read .eea and .edf files, compute the PSD, SE, and DE features, and create a 2D brain map using a predefined channel-to-position map.
"""

'\n## 1. Functions for Reading Files and Computing Features\nThese functions read .eea and .edf files, compute the PSD, SE, and DE features, and create a 2D brain map using a predefined channel-to-position map.\n'

In [18]:
def read_eea_file(eea_file, num_channels=16, samples_per_channel=7680):
    with open(eea_file, 'r') as f:
        data = f.read().split('\n')
    data = [float(item) for item in data if item]
    return np.reshape(data, (num_channels, samples_per_channel))

def compute_psd(signal, fs=128, nperseg=256):
    freqs, psd = welch(signal, fs=fs, nperseg=nperseg)
    return freqs, psd

def compute_se(psd, freqs, bands):
    se = []
    for band in bands:
        band_mask = (freqs >= band[0]) & (freqs <= band[1])
        band_psd = psd[band_mask]
        total = np.sum(band_psd)
        if total != 0:
            band_psd = band_psd / total
        se.append(entropy(band_psd))
    return se

def compute_de(psd, freqs, bands):
    de = []
    for band in bands:
        band_mask = (freqs >= band[0]) & (freqs <= band[1])
        band_psd = psd[band_mask]
        band_de = np.sum(np.log(band_psd + 1e-10) * band_psd)
        de.append(band_de)
    return de

def extract_features(signal, fs=128, bands=None):
    if bands is None:
        bands = [(4,8), (8,12), (13,30), (30,40)]  # theta, alpha, beta, gamma
    freqs, psd = compute_psd(signal, fs)
    psd_features = np.array([np.trapz(psd[(freqs >= b[0]) & (freqs <= b[1])]) for b in bands])
    se_features = compute_se(psd, freqs, bands)
    de_features = compute_de(psd, freqs, bands)
    return psd_features, se_features, de_features

def create_brain_map_per_band(feature_data, channels, position_map):
    """
    feature_data: 1D array with one feature per channel for a given band.
    """
    brain_map = np.zeros((8, 9))
    for ch in channels:
        if ch in position_map:
            pos = position_map[ch]
            x, y = map(int, pos.split(','))
            brain_map[x, y] = feature_data[channels.index(ch)]
    return brain_map

# %% [markdown]
"""
### Reading .edf Files
We use pyedflib to read .edf files. Channels 'Fp1', 'Fz', and 'Fp2' are dropped if present.
"""

"\n### Reading .edf Files\nWe use pyedflib to read .edf files. Channels 'Fp1', 'Fz', and 'Fp2' are dropped if present.\n"

In [19]:
def read_edf(edf_file):
    f = pyedflib.EdfReader(edf_file)
    n = f.signals_in_file
    signal_labels = f.getSignalLabels()
    sigbufs = np.zeros((n, f.getNSamples()[0]))
    for i in range(n):
        sigbufs[i, :] = f.readSignal(i)
    df = pd.DataFrame(sigbufs.transpose(), columns=signal_labels)
    for ch in ['Fp1', 'Fz', 'Fp2']:
        if ch in df.columns:
            df = df.drop(columns=[ch])
    return df

# %% [markdown]
"""
### Processing Functions for .eea and .edf Files
For .eea files, we process files from two subdirectories (with labels 0 and 1) and create brain maps per band.
For .edf files, we label based on the filename prefix ('h' for class 0, 's' for class 1) and create one brain map per band.
"""

"\n### Processing Functions for .eea and .edf Files\nFor .eea files, we process files from two subdirectories (with labels 0 and 1) and create brain maps per band.\nFor .edf files, we label based on the filename prefix ('h' for class 0, 's' for class 1) and create one brain map per band.\n"

In [20]:
def process_eea_and_store_brain_maps(directory, channels, position_map, bands=[(4,8), (8,12), (13,30), (30,100)]):
    all_psd_maps = []
    all_se_maps = []
    all_de_maps = []
    labels = []
    for subdir, label in [('SCZ-Dataset2-Normal', 0), ('SCZ-Dataset2-SCZ', 1)]:
        subdir_path = os.path.join(directory, subdir)
        for filename in os.listdir(subdir_path):
            if filename.endswith(".eea"):
                file_path = os.path.join(subdir_path, filename)
                data_array = read_eea_file(file_path)
                psd_per_channel = []
                se_per_channel = []
                de_per_channel = []
                for ch in range(data_array.shape[0]):
                    psd_feat, se_feat, de_feat = extract_features(data_array[ch, :], fs=128, bands=bands)
                    psd_per_channel.append(psd_feat)
                    se_per_channel.append(se_feat)
                    de_per_channel.append(de_feat)
                psd_per_band = np.transpose(np.array(psd_per_channel))  # (num_bands, num_channels)
                se_per_band  = np.transpose(np.array(se_per_channel))
                de_per_band  = np.transpose(np.array(de_per_channel))
                psd_maps = np.array([create_brain_map_per_band(psd_per_band[i], channels, position_map) for i in range(len(bands))])
                se_maps  = np.array([create_brain_map_per_band(se_per_band[i], channels, position_map) for i in range(len(bands))])
                de_maps  = np.array([create_brain_map_per_band(de_per_band[i], channels, position_map) for i in range(len(bands))])
                all_psd_maps.append(psd_maps)
                all_se_maps.append(se_maps)
                all_de_maps.append(de_maps)
                labels.append(label)
    return (np.array(all_psd_maps), np.array(all_se_maps), np.array(all_de_maps)), np.array(labels)

def process_and_store_brain_maps(directory, channels, position_map, bands=[(4,8), (8,12), (13,30), (30,100)]):
    all_psd_maps = []
    all_se_maps = []
    all_de_maps = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".edf"):
            if filename.startswith("h"):
                label = 0
            elif filename.startswith("s"):
                label = 1
            else:
                continue
            df = read_edf(os.path.join(directory, filename))
            data_array = df[channels].values  # shape: (num_samples, num_channels)
            psd_brain_maps_per_band = []
            se_brain_maps_per_band = []
            de_brain_maps_per_band = []
            for ch in range(data_array.shape[1]):
                psd, se, de = extract_features(data_array[:, ch], fs=256, bands=bands)
                psd_brain_maps_per_band.append(psd)
                se_brain_maps_per_band.append(se)
                de_brain_maps_per_band.append(de)
            trans_psd = np.transpose(np.array(psd_brain_maps_per_band))  # (num_bands, num_channels)
            trans_se  = np.transpose(np.array(se_brain_maps_per_band))
            trans_de  = np.transpose(np.array(de_brain_maps_per_band))
            psd_maps = np.array([create_brain_map_per_band(trans_psd[i], channels, position_map) for i in range(len(bands))])
            se_maps  = np.array([create_brain_map_per_band(trans_se[i], channels, position_map) for i in range(len(bands))])
            de_maps  = np.array([create_brain_map_per_band(trans_de[i], channels, position_map) for i in range(len(bands))])
            all_psd_maps.append(psd_maps)
            all_se_maps.append(se_maps)
            all_de_maps.append(de_maps)
            labels.append(label)
    return (np.array(all_psd_maps), np.array(all_se_maps), np.array(all_de_maps)), np.array(labels)

# %% [markdown]
"""
## 2. Data Preparation: Merging .eea and .edf Datasets and Augmentation
We process .eea and .edf files from separate directories, merge the datasets, and then augment.
- First, augment class 1 with n_augments=4 per sample.
- Then, compute current class counts and determine how many additional class 0 samples are needed so that the final ratio of class1:class0 becomes 1:2.
- Augment class 0 accordingly.
"""

'\n## 2. Data Preparation: Merging .eea and .edf Datasets and Augmentation\nWe process .eea and .edf files from separate directories, merge the datasets, and then augment.\n- First, augment class 1 with n_augments=4 per sample.\n- Then, compute current class counts and determine how many additional class 0 samples are needed so that the final ratio of class1:class0 becomes 1:2.\n- Augment class 0 accordingly.\n'

In [21]:
# Define channels and electrode position map
channels = ['F7','F3','F4','F8','T3','C3','Cz','C4','T4','T5','P3','Pz','P4','T6','O1','O2']
position_map = {
    'F7':'1,0', 'F3':'1,2', 'F4':'1,6', 'F8':'1,8',
    'T3':'3,0', 'C3':'3,2', 'Cz':'3,4', 'C4':'3,6', 'T4':'3,8',
    'T5':'5,0', 'P3':'5,2', 'Pz':'5,4', 'P4':'5,6', 'T6':'5,8',
    'O1':'7,3', 'O2':'7,5'
}

# Process .eea files (e.g., from "mhrc-dataset-data")
eea_directory = "/kaggle/input/mhrc-dataset-data"
(all_psd_maps_eea, all_se_maps_eea, all_de_maps_eea), labels_eea = process_eea_and_store_brain_maps(eea_directory, channels, position_map)

# Process .edf files (e.g., from "schizophrenia")
edf_directory = "/kaggle/input/schizophrenia"
(all_psd_maps_edf, all_se_maps_edf, all_de_maps_edf), labels_edf = process_and_store_brain_maps(edf_directory, channels, position_map)

print("EEA dataset shapes (PSD,SE,DE):", all_psd_maps_eea.shape, all_se_maps_eea.shape, all_de_maps_eea.shape)
print("EDF dataset shapes (PSD,SE,DE):", all_psd_maps_edf.shape, all_se_maps_edf.shape, all_de_maps_edf.shape)

# Merge the two datasets (order: EDF then EEA)
merged_psd_maps = np.concatenate([all_psd_maps_edf, all_psd_maps_eea], axis=0)
merged_se_maps  = np.concatenate([all_se_maps_edf, all_se_maps_eea], axis=0)
merged_de_maps  = np.concatenate([all_de_maps_edf, all_de_maps_eea], axis=0)
merged_labels   = np.concatenate([labels_edf, labels_eea], axis=0)

print("Merged PSD maps shape:", merged_psd_maps.shape)
print("Merged Labels shape:", merged_labels.shape)

EEA dataset shapes (PSD,SE,DE): (84, 4, 8, 9) (84, 4, 8, 9) (84, 4, 8, 9)
EDF dataset shapes (PSD,SE,DE): (28, 4, 8, 9) (28, 4, 8, 9) (28, 4, 8, 9)
Merged PSD maps shape: (112, 4, 8, 9)
Merged Labels shape: (112,)


In [22]:
def augment_brain_maps_specific_class(brain_maps, labels, target_class, n_augments=4, random_seed=42):
    np.random.seed(random_seed)
    augmented_maps = []
    augmented_labels = []
    for i in range(len(labels)):
        if labels[i] == target_class:
            possible_indices = [j for j, lab in enumerate(labels) if lab == target_class and j != i]
            for _ in range(n_augments):
                if len(possible_indices) == 0:
                    continue
                j = np.random.choice(possible_indices)
                alpha = np.random.rand()
                new_sample = alpha * brain_maps[i] + (1 - alpha) * brain_maps[j]
                augmented_maps.append(new_sample)
                augmented_labels.append(target_class)
    if len(augmented_maps) > 0:
        augmented_maps = np.array(augmented_maps)
        augmented_labels = np.array(augmented_labels)
        brain_maps_aug = np.concatenate([brain_maps, augmented_maps], axis=0)
        labels_aug = np.concatenate([labels, augmented_labels], axis=0)
        return brain_maps_aug, labels_aug
    else:
        return brain_maps, labels

def augment_class_to_target(brain_maps, labels, target_class, additional_needed, random_seed=42):
    np.random.seed(random_seed)
    class_indices = [i for i, lab in enumerate(labels) if lab == target_class]
    augmented_samples = []
    for _ in range(additional_needed):
        i, j = np.random.choice(class_indices, size=2, replace=True)
        alpha = np.random.rand()
        new_sample = alpha * brain_maps[i] + (1 - alpha) * brain_maps[j]
        augmented_samples.append(new_sample)
    if len(augmented_samples) > 0:
        augmented_samples = np.array(augmented_samples)
        new_labels = np.full((augmented_samples.shape[0],), target_class)
        brain_maps_aug = np.concatenate([brain_maps, augmented_samples], axis=0)
        labels_aug = np.concatenate([labels, new_labels], axis=0)
        return brain_maps_aug, labels_aug
    else:
        return brain_maps, labels

# First, augment target class 1 (n_augments=4 per sample) for each feature type.
merged_psd_maps_aug, merged_labels_aug = augment_brain_maps_specific_class(merged_psd_maps, merged_labels, target_class=1, n_augments=4)
merged_se_maps_aug, merged_labels_aug = augment_brain_maps_specific_class(merged_se_maps, merged_labels_aug, target_class=1, n_augments=4)
merged_de_maps_aug, merged_labels_aug = augment_brain_maps_specific_class(merged_de_maps, merged_labels_aug, target_class=1, n_augments=4)

# Compute current counts.
count_class1 = np.sum(merged_labels_aug == 1)
count_class0 = np.sum(merged_labels_aug == 0)
desired_class0 = 2 * count_class1
additional_needed = desired_class0 - count_class0
print("After class 1 augmentation: Class0 =", count_class0, ", Class1 =", count_class1)
print("Additional class 0 samples needed for 1:2 ratio:", additional_needed)

# Augment class 0 if additional samples are needed.
if additional_needed > 0:
    merged_psd_maps_aug, merged_labels_aug = augment_class_to_target(merged_psd_maps_aug, merged_labels_aug, target_class=0, additional_needed=additional_needed, random_seed=42)
    merged_se_maps_aug, merged_labels_aug = augment_class_to_target(merged_se_maps_aug, merged_labels_aug, target_class=0, additional_needed=additional_needed, random_seed=42)
    merged_de_maps_aug, merged_labels_aug = augment_class_to_target(merged_de_maps_aug, merged_labels_aug, target_class=0, additional_needed=additional_needed, random_seed=42)

print("After augmentation, PSD maps shape:", merged_psd_maps_aug.shape)
print("After augmentation, Labels shape:", merged_labels_aug.shape)

# %% [markdown]
"""
## 3. Stacking Feature Maps for ChronoNet Input
For each sample and each band (4 bands), the 8×9 brain maps (for PSD, SE, and DE) are flattened (72 elements each)
and concatenated to form a 216-dimensional vector per band. The final input shape becomes (num_samples, 4, 216).
"""

IndexError: index 156 is out of bounds for axis 0 with size 112

In [ ]:
def stack_maps_for_chrononet(psd_maps, se_maps, de_maps):
    num_samples, num_bands, h, w = psd_maps.shape
    stacked = []
    for i in range(num_samples):
        sample_seq = []
        for b in range(num_bands):
            psd_flat = psd_maps[i, b].flatten()
            se_flat  = se_maps[i, b].flatten()
            de_flat  = de_maps[i, b].flatten()
            combined = np.concatenate([psd_flat, se_flat, de_flat])  # 72 * 3 = 216
            sample_seq.append(combined)
        stacked.append(np.array(sample_seq))
    return np.array(stacked)

stacked_features = stack_maps_for_chrononet(merged_psd_maps_aug, merged_se_maps_aug, merged_de_maps_aug)
print("Stacked features shape (ChronoNet input):", stacked_features.shape)

# One-hot encode labels (2 classes)
labels_cat = to_categorical(merged_labels_aug, num_classes=2)

# %% [markdown]
"""
## 4. Building a ChronoNet-Inspired Model
The model takes input of shape (4, 216) and uses parallel 1D convolution layers (with kernel sizes 2, 4, and 8).
Their outputs are concatenated, followed by another parallel convolution block, two GRU layers, dropout,
and a dense softmax output layer.
"""

In [ ]:
def build_chrononet_model(input_shape=(4, 216), num_classes=2):
    inputs = Input(shape=input_shape, name='input_layer')
    conv_outputs = []
    for k in [2, 4, 8]:
        conv = layers.Conv1D(filters=32, kernel_size=k, strides=1, padding='same', activation='relu')(inputs)
        conv_outputs.append(conv)
    x = layers.Concatenate(name='concat_conv1')(conv_outputs)
    conv_outputs = []
    for k in [2, 4, 8]:
        conv = layers.Conv1D(filters=32, kernel_size=k, strides=2, padding='same', activation='relu')(x)
        conv_outputs.append(conv)
    x = layers.Concatenate(name='concat_conv2')(conv_outputs)
    gru1 = layers.GRU(128, return_sequences=True, name='gru1')(x)
    gru2 = layers.GRU(128, return_sequences=False, name='gru2')(gru1)
    x = layers.Dropout(0.5)(gru2)
    outputs = layers.Dense(num_classes, activation='softmax', name='output')(x)
    model = models.Model(inputs=inputs, outputs=outputs, name='ChronoNet_Model')
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

chrononet_model = build_chrononet_model(input_shape=(4,216), num_classes=2)
chrononet_model.summary()
plot_model(chrononet_model, to_file='chrononet_model_architecture.png', show_shapes=True, show_layer_names=True, dpi=100)

# %% [markdown]
"""
## 5. Training the ChronoNet Model
We split the stacked features and one-hot labels into training and validation sets, then train the model with early stopping
and learning rate reduction.
"""

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(stacked_features, labels_cat, test_size=0.2, random_state=42)

early_stopping_cb = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history = chrononet_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping_cb, reduce_lr_cb]
)

chrononet_model.save('/kaggle/working/chrononet_model.h5')

# %% [markdown]
"""
## 6. Plotting Training Metrics and Evaluation
We plot training/validation loss and accuracy, and generate a confusion matrix and classification report on the validation set.
"""

In [ ]:
def plot_training_validation_loss(history):
    plt.figure(figsize=(14,6))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_training_validation_loss(history)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = chrononet_model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)
cm = confusion_matrix(y_true, y_pred_classes)
cr = classification_report(y_true, y_pred_classes, target_names=['Class 0','Class 1'])

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0','Class 1'], yticklabels=['Class 0','Class 1'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

print("Classification Report:\n", cr)
